In [1]:
import torch
import numpy as np
import pinot

Using backend: pytorch


In [2]:
ds = pinot.data.esol()
# ds = pinot.data.utils.batch(ds, 8)
ds_tr, ds_te = pinot.data.utils.split(ds, [2, 1])
ds_tr = pinot.data.utils.batch(ds_tr, len(ds_tr))
ds_te = pinot.data.utils.batch(ds_te, len(ds_te))

In [3]:
graph_kernel = pinot.inference.gp.deep_kernel.DeepKernel(
    representation=pinot.representation.Sequential(
        pinot.representation.dgl_legacy.gn(),
        [32, 'tanh', 32, 'tanh', 32, 'tanh']),
    base_kernel=pinot.inference.gp.kernels.RBF())

In [4]:
opt = torch.optim.Adam(graph_kernel.parameters(), 1e-3)
x_tr, y_tr = ds_tr[0]
for _ in range(1):
    opt.zero_grad()
    loss = graph_kernel.loss(x_tr, y_tr).sum()
    loss.backward()
    print(loss)
    opt.step()

/Users/wangy1/anaconda3/envs/pinot/lib/python3.7/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[1.0000e+00, 4.2608e-03, 6.8452e-04,  ..., 1.1114e-02, 8.6109e-03,
         1.7227e-05],
        [4.2608e-03, 1.0000e+00, 9.9056e-02,  ..., 1.5708e-04, 8.0134e-05,
         2.7323e-03],
        [6.8452e-04, 9.9056e-02, 1.0000e+00,  ..., 1.8280e-05, 1.0482e-05,
         2.2981e-02],
        ...,
        [1.1114e-02, 1.5708e-04, 1.8280e-05,  ..., 1.0000e+00, 2.4919e-01,
         4.4090e-07],
        [8.6109e-03, 8.0134e-05, 1.0482e-05,  ..., 2.4919e-01, 1.0000e+00,
         2.4591e-07],
        [1.7227e-05, 2.7323e-03, 2.2981e-02,  ..., 4.4090e-07, 2.4591e-07,
         1.0000e+00]], grad_fn=<ExpBackward>)
tensor([[ 5.2950e-01, -1.8380e-06,  1.2209e-08,  ...,  1.2646e-05,
         -1.8897e-06,  2.4682e-09],
        [-1.8383e-06,  7.6454e-01,  4.3368e-05,  ..., -3.8444e-08,
          4.7521e-09,  7.7528e-07],
        [ 1.2251e-08,  4.3357e-05,  7.0293e-01,  ..., -6.1490e-09,
         -6.2191e-09,  3.4004e-05],
        ...,
        [ 1.2645e-05, -3.8434e-08, -6.1495e-09,  ...,  6.32

In [21]:
x_te, y_te = ds_te[0]
distribution = graph_kernel.inference(x_te, y_te, x_te)

RuntimeError: cholesky_cpu: U(1,1) is zero, singular U.

In [ ]:
distribution.log_prob(y_te.to(dtype=torch.float32))

In [ ]:
distribution.mean

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt

plt.scatter(y_tr.detach().numpy(), distribution.mean.detach().numpy())